In [1]:
import json
import random
import jinja2
import os
import os.path
import subprocess
from latools import *
from sympy import *
init_printing(use_latex=True)

# Function Definitions

In [304]:
def rref_to_solution_basis(R):
    m, n = R.shape
    nvars = n - 1
    jold = -1
    pivots = []
    zero_rows = []
    for i in range(m):
        for j in range(n-1):
            if R[i,j] != 0:
                break
        else:
            zero_rows.append(i)
            continue
        if len(zero_rows) != 0:
            raise ValueError('zero rows not at the bottom')
        if R[i,j] != 1:
            raise ValueError('leading element of row {} is not 1'.format(i+1))
        for k in range(m):
            if k != i and R[k, j] != 0:
                raise ValueError('nonzero in pivot column {}'.format(j+1))
        if j <= jold:
            raise ValueError('pivot columns not in increasing order')
        pivots.append((i,j))
    for i in zero_rows:
        if R[i,nvars] != 0:
            return None, None
    pivot_columns = [j for _ , j in pivots]
    non_pivot_columns = [j for j in range(n-1) if j not in pivot_columns]
    cvec = zeros(nvars, 1)
    for (i,j) in pivots:
        cvec[j] = R[i, n-1]
    bvecs = [zeros(nvars,1) for _ in range(len(non_pivot_columns))]
    for i, j in enumerate(non_pivot_columns):
        bvecs[i][j] = 1
        for k, l in pivots:
            bvecs[i][l] = -R[k,j]
    return cvec, bvecs

def vector_to_latex(vec, align=''):
    if len(align) > 0:
        start = '\\begin{{bmatrix*}}[{}]'.format(align)
        end =  '\\end{bmatrix*}'
    else:
        start = '\\begin{bmatrix}'
        end = '\\end{bmatrix}'
    return start + '\\\\'.join(str(v) for v in vec) + end

def rref_to_solution_latex(R, align=''):
    cvec, bvecs = rref_to_solution_basis(R)
    if cvec is None:
        return '\\varnothing'
    sout = '\\left\\{' + vector_to_latex(cvec, align=align) + '\n' 
    for i, vec in enumerate(bvecs):
        sout += '+\\lambda_{{{}}}'.format(i+1) + vector_to_latex(vec, align=align) + '\n'
    sout += '\\;:\\;'
    sout += ','.join('\\lambda_{{{}}}'.format(i+1) for i in range(len(bvecs)))
    sout += '\\in\\mathbb{R}\\right\\}\n'
    return sout

def make_linear_system_exercise(R, pvalues=range(-4,4), align='', vnames=None):
    m, n = R.shape
    P = zeros(m)
    while P.det() == 0:
        for i in range(m):
            for j in range(m):
                P[i, j] = nsimplify(random.choice(pvalues))
    A = P * R
    system_str = matrix_to_system_latex(A, vnames)
    solution_str = rref_to_solution_latex(R, align=align)
    return system_str, solution_str

def pattern_to_exercise(pattern, rvalues=range(-5,5), pvalues=range(-3,3), align=''):
    m = len(pattern)
    n = max(len(pat) for pat in pattern)
    R = zeros(m, n)
    for i, pat in enumerate(pattern):
        for j, s in enumerate(pat):
            R[i,j] = 0 if s == '0' else 1 if s == '1' else random.choice(rvalues)
    return make_linear_system_exercise(R, pvalues, align=align)

def pattern_to_rref(pattern, rvalues=range(-5,5)):
    m = len(pattern)
    n = max(len(pat) for pat in pattern)
    R = zeros(m, n)
    for i, pat in enumerate(pattern):
        for j, s in enumerate(pat):
            R[i,j] = 0 if s == '0' else 1 if s == '1' else random.choice(rvalues)
    return R

def make_matrix_inversion_exercise(n, cmax=3):
    A = eye(n)
    cvalues = [k for k in range(-cmax, cmax+1) if k != 0]
    nrows, _ = A.shape
    for i in range(nrows):
        for j in range(nrows):
            if i == j:
                continue
            c = random.choice(cvalues)
            A[j,:] += c*A[i,:]
            c = random.choice(cvalues)
            A[:,j] += c*A[:,i]
    return matrix_to_latex(A)

def gram_schmidt(ulist):
    vlist = [ulist[0]]
    n, _ = ulist[0].shape
    for u in ulist[1:]:
        vlist.append(u - sum([u.dot(v)/v.dot(v)*v for v in vlist], zeros(n, 1)))
    return vlist

def random_invertible_matrix(n, cvalues=[-1,1], determinant=1):
    P = eye(n)
    P[0,:] *= determinant
    for i in range(n):
        for j in range(n):
            if i==j:
                continue
            P[j,:] += random.choice(cvalues)*P[i,:]
            P[:,j] += random.choice(cvalues)*P[:,i]        
    return P

def vector_to_latex(v, align='r'):
    strout = '\\begin{{bmatrix*}}[{}]'.format(align)
    for elem in v:
        strout += '{}\\\\ '.format(latex(elem))
    strout +='\\end{bmatrix*}'
    return strout

def matrix_to_latex(A, align='r'):
    n, m = A.shape
    strout = '\\begin{{bmatrix*}}[{}]'.format(align)
    for i in range(n):
        strout += '&'.join([latex(entry) for entry in A[i,:]]) + '\\\\'
    strout += '\\end{bmatrix*}'
    return strout

# Setup

In [305]:
latex_jinja_env = jinja2.Environment(
    block_start_string = '\BLOCK{',
    block_end_string = '}',
    variable_start_string = '\VAR{',
    variable_end_string = '}',
    comment_start_string = '\#{',
    comment_end_string = '}',
    line_statement_prefix = '%%',
    line_comment_prefix = '%#',
    trim_blocks = True,
    autoescape = False,
    loader = jinja2.FileSystemLoader(os.path.abspath('.'))
)
random.seed(11111111)
number_of_versions = 4
version_symbols = ['\\diamondsuit', '\\heartsuit', '\\clubsuit', '\\spadesuit']
file_prefix = 'final-fall2016-'
file_suffix = '.tex'
output_dir = 'versions'
pdf_command = 'pdflatex'

# Problem 1: Linear System

In [306]:
patterns = [["10x00xx",
             "01x00xx",
             "00010xx",
             "0000100"],
            ["1x000xx",
             "00100xx",
             "00010xx",
             "00001xx"],
            ["1x00x0x",
             "0010x0x",
             "0001x0x",
             "0000010"],
            ["10x00xx",
             "01x00xx",
             "00010xx",
             "00001xx"]
           ]

In [307]:
problem1_versions = []
for pattern in patterns:
    print(pattern)
    problem, solution = pattern_to_exercise(pattern)
    problem1_versions.append(problem)

['10x00xx', '01x00xx', '00010xx', '0000100']
['1x000xx', '00100xx', '00010xx', '00001xx']
['1x00x0x', '0010x0x', '0001x0x', '0000010']
['10x00xx', '01x00xx', '00010xx', '00001xx']


# Problem 2: Matrix Inversion

In [308]:
problem2_versions = []
for _ in range(number_of_versions):
    problem = make_matrix_inversion_exercise(4, cmax=1)
    problem2_versions.append(problem)

# Problem 3: Singular Matrix

In [309]:
problem3_versions = []
for _ in range(number_of_versions):
    lbound = 5
    lvalues = [v for v in range(-lbound, lbound) if v != 0]
    lambda1 = random.choice(lvalues)
    lambda2 = lambda1
    while lambda2 == lambda1:
        lambda2 = random.choice(lvalues)
    k = symbols('k')
    A = matrix_to_rational([[k, lambda1, 1],[lambda1,k,1],[lambda1,lambda2,1]])
    cvalues = [-3, -2, -1, 2, 3]
    A[:,2] *= random.choice(cvalues)
    A[2,:] *= random.choice(cvalues)
    j = random.choice([1,2])
    A[:,0], A[:,j] = A[:,j], A[:,0]
    A[:,1], A[:,2] = A[:,2], A[:,1]
    if random.choice([True,False]):
        A[2,:], A[1,:] = A[1,:], A[2,:]
    problem3_versions.append(matrix_to_latex(A))

# Problem 4: Change of Basis

In [310]:
problem4_versions = []
for _ in range(number_of_versions):
    A = random_invertible_matrix(4, cvalues=[-1,1], determinant=1)
    vlist = [A[:,j] for j in range(4)]
    cvalues = range(-2,3)
    v = sum([random.choice(cvalues)*v for v in vlist],ones(4,1))
    vlist.append(v)
    problem4_versions.append([vector_to_latex(v) for v in vlist])

# Problem 5: Range and Kernel

In [311]:
patterns = [["1x0xx",
             "001xx",
             "00000",
             "00000"],
            ["10x0x",
             "01x0x",
             "00010",
             "00000"],             
            ["1xx0x",
             "0001x",
             "00000",
             "00000"],
            ["10x0x",
             "01x0x",
             "00010",
             "00000"],
           ]

In [312]:
problem5_versions=[]
for pattern in patterns:
    R = pattern_to_rref(pattern)
    m, n = R.shape
    P = zeros(m)
    pvalues=range(-3,3)
    while P.det() == 0:
        for i in range(m):
            for j in range(m):
                P[i, j] = nsimplify(random.choice(pvalues))
    A = P * R
    problem5_versions.append(matrix_to_latex(A))

# Problem 6: Diagonalize Symmetric Matrix

In [386]:
v1 = Matrix([1,0,1,0])
v2 = Matrix([0,1,0,1])
v3 = Matrix([1,1,-1,-1])
v4 = Matrix([1,-1,-1,1])
P=Matrix.hstack(v1,v2,v3,v4)
dvals = [-3,-2,-1,1,2,3]
problem6_versions=[]
for _ in range(4):
    a = random.choice(dvals)
    b = a
    while b==a:
        b = random.choice(dvals)
    D = diag(a,a,a,b)
    A = 4 * P * D * P**(-1)
    problem6_versions.append(matrix_to_latex(A))

# Problem 8: Properties of dot product

In [400]:
problem8_versions = [random.choice([2,3,4,5]) for _ in range(4)]

# Setup

In [401]:
latex_jinja_env = jinja2.Environment(
    block_start_string = '\BLOCK{',
    block_end_string = '}',
    variable_start_string = '\VAR{',
    variable_end_string = '}',
    comment_start_string = '\#{',
    comment_end_string = '}',
    line_statement_prefix = '%%',
    line_comment_prefix = '%#',
    trim_blocks = True,
    autoescape = False,
    loader = jinja2.FileSystemLoader(os.path.abspath('.'))
)

# Rendering

In [404]:
template = latex_jinja_env.get_template('{}template{}'.format(file_prefix, file_suffix))
for version_number in range(number_of_versions):
    print('Generating version {}'.format(version_number+1))
    template_dict = {}
    template_dict['version_symbol'] = version_symbols[version_number]
    template_dict['problem1'] = problem1_versions[version_number]
    template_dict['problem2'] = problem2_versions[version_number]
    template_dict['problem3'] = problem3_versions[version_number]
    template_dict['problem4_v1'] = problem4_versions[version_number][0]
    template_dict['problem4_v2'] = problem4_versions[version_number][1]
    template_dict['problem4_v3'] = problem4_versions[version_number][2]
    template_dict['problem4_v4'] = problem4_versions[version_number][3]
    template_dict['problem4_v5'] = problem4_versions[version_number][4]
    template_dict['problem5'] = problem5_versions[version_number]
    template_dict['problem6'] = problem6_versions[version_number]
    template_dict['problem8'] = problem8_versions[version_number]
    sout = template.render(**template_dict)
    output_file_name = '{}version{}{}'.format(file_prefix, version_number + 1, file_suffix)
    current_dir = os.getcwd()
    os.chdir(os.path.join('.', output_dir))
    fout = open(output_file_name, 'w')
    fout.write(sout)
    fout.close()
    completed_process = subprocess.run([pdf_command, output_file_name], stdout=subprocess.PIPE)
    os.chdir(current_dir)

Generating version 1
Generating version 2
Generating version 3
Generating version 4
